<a href="https://colab.research.google.com/github/JRKagumba/2D-video-based-running-analysis/blob/main/notebooks/00_Apply_OpenPose_to_Raw_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Import Libraries

In [2]:
import os
import glob
import subprocess
import time

import pandas as pd
import numpy as np
import json

import shutil

### Install cmake and OpenPose dependencies

In [3]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

### Initialize Helper Functions and Data Structures

In [4]:
def convert_json2csv(json_directory, activity_folder):
    # determine the number of frames
    nframes = len(os.listdir(json_directory))

    # initialize res to be array of NaN
    res = np.zeros((nframes,75))
    res[:] = np.nan

    PROJ_JSON_ROOT = os.path.join('/content/openpose/output/')

    # read in JSON files
    for frame in range(0,nframes):
        
        test_image_json = os.path.join(PROJ_JSON_ROOT, f'{activity_folder}_{str(frame).zfill(12)}_keypoints.json')

        # if not os.path.isfile(test_image_json):
        #     break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)

        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            res[frame,:] = keypoints
            break

    return res


keywords_dict = {
       0 : 'NOSE',  1 : 'NECK',  2 : 'RSHO',  3 : 'RELB',  4 : 'RWRI',
       5 : 'LSHO',  6 : 'LELB',  7 : 'LWRI',  8 : 'MHIP',  9 : 'RHIP',
      10 : 'RKNE', 11 : 'RANK', 12 : 'LHIP', 13 : 'LKNE', 14 : 'LANK',
      15 : 'REYE', 16 : 'LEYE', 17 : 'REAR', 18 : 'LEAR', 19 : 'LBTO',
      20 : 'LSTO', 21 : 'LHEL', 22 : 'RBTO', 23 : 'RSTO', 24 : 'RHEL'
      }

def dataframe_preprocessing(df):
    """
    Rename columns and remove probability column
    """
    name_replace_dict={}

    for col in df.columns:
        axis = (int(col)) % 3

        axis_dict = {0:'X', 1:'Y', 2:'Prob'} 
        axis_var = axis_dict[axis]

        keypoint = (int(col)) // 3
        keypoint_var = keywords_dict[keypoint]

        name_replace_dict[col] = f'{keypoint_var}_{axis_var}'


    df = df.rename(columns=name_replace_dict)
    df = df[df.columns.drop(list(df.filter(regex='Prob')))]

    return df

### Apply OpenPose to Raw Videos and save Time Series data

In [5]:
PROJ_SRC_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/raw')
PROJ_SAVE_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/processed')

position_lst = os.listdir(PROJ_SRC_ROOT)
run_log_data = {}

for position in position_lst:
    raw_video_lst = os.listdir(os.path.join(PROJ_SRC_ROOT, position))

    print(position)
    for raw_video_file in raw_video_lst:

        src_file_path = os.path.join(PROJ_SRC_ROOT, position, raw_video_file)
        raw_video_folder = raw_video_file[:-4]

        print(f'\t{raw_video_folder:20}')

        #Instantiate list for each activity
        run_log_data[position] = []
        
        #Append file size to run logs
        file_size = os.path.getsize(src_file_path)
        run_log_data[position].append(file_size)

        #Clear json files from previous run
        for f in glob.glob('/content/openpose/output/*.json'):
            os.remove(f)

        #Clear avi files from previous run 
        subprocess.run(["rm", "openpose.avi"])

        #Run Pose Detection
        start = time.time()
        os.chdir('/content/openpose')
        subprocess.run(["./build/examples/openpose/openpose.bin", 
                "--video", src_file_path, 
                "--write_json", "./output/", 
                "--display", "0",  
                "--write_video", "../openpose.avi"], 
                shell=False)
        end = time.time()
        pose_detection_time = round(end-start, 1)
        run_log_data[position].append(pose_detection_time)  

        print(f'\t\t Pose Detected in {pose_detection_time:5}s')

        #AVI to MP4 conversion
        start = time.time()
        os.chdir('/content')
        subprocess.run(["ffmpeg", "-y", "-loglevel", "info", "-i", "openpose.avi", "output.mp4"], shell=False)
        end = time.time()
        video_convert_time = round(end-start, 1)
        run_log_data[position].append(video_convert_time) 

        print(f'\t\t Video Converted in {video_convert_time:5}s') 

        #Create csv output from json files
        biomech_df = pd.DataFrame(convert_json2csv("/content/openpose/output/", raw_video_folder))

        #Apply pre-processing steps
        biomech_df = dataframe_preprocessing(biomech_df)

        #########################################  SAVING  ##########################################

        # Make specific folder for activity files
        path = os.path.join(PROJ_SAVE_ROOT, position, raw_video_folder)

        if os.path.exists(path):
            pass
        else:
            os.mkdir(path)

        #### DATA
        data_save_path = os.path.join(PROJ_SAVE_ROOT, position, raw_video_folder, f"{raw_video_folder}.csv")
        biomech_df.to_csv(data_save_path)

        print(f'\t\t Save path : {data_save_path:20}')

        #### VIDEOS
        avi_video_src_file_path = '/content/openpose.avi'
        mp4_video_src_file_path = '/content/output.mp4'

        avi_save_path = os.path.join(PROJ_SAVE_ROOT, position, raw_video_folder, f"{raw_video_folder}.avi")
        mp4_save_path = os.path.join(PROJ_SAVE_ROOT, position, raw_video_folder, f"{raw_video_folder}.mp4")

        shutil.move(avi_video_src_file_path, avi_save_path)
        shutil.move(mp4_video_src_file_path, mp4_save_path)

        print(f'\t\t Video Saved') 


defensive_line
	defensive_line_01   
		 Pose Detected in  15.2s
		 Video Converted in   3.3s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/processed/defensive_line/defensive_line_01/defensive_line_01.csv
		 Video Saved
	defensive_line_02   
		 Pose Detected in  10.4s
		 Video Converted in   3.3s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/processed/defensive_line/defensive_line_02/defensive_line_02.csv
		 Video Saved
	defensive_line_03   
		 Pose Detected in  10.1s
		 Video Converted in   3.5s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/processed/defensive_line/defensive_line_03/defensive_line_03.csv
		 Video Saved
	defensive_line_04   
		 Pose Detected in  12.4s
		 Video Converted in   4.0s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___40_YARD_DASH/data/processed/defensive_line/defensive_line_04/defens

In [6]:
biomech_df = pd.DataFrame.from_dict(run_log_data, orient='index', columns=['File_Size', "Pose_Detection_Time(s)", "Video_Convert_Time(s)"])

biomech_df

,File_Size,Pose_Detection_Time(s),Video_Convert_Time(s)
defensive_line,3035898,10.5,3.6
offensive_line,3071622,10.6,3.2
wide_receivers,3193314,10.7,3.5
quarterbacks,2753779,9.8,2.9
running_backs,2713047,10.3,3.1
defensive_back,2733890,10.2,3.0
